In [1]:
from sklearn.feature_selection import SelectKBest,RFE
from mlxtend.feature_selection import SequentialFeatureSelector as SFS
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

In [2]:
import pandas as pd
dataset=pd.read_csv("CKD.csv")
dataset=pd.get_dummies(dataset,drop_first=True)
indep=dataset.drop('classification_yes',axis=1)
dep=dataset['classification_yes']

In [3]:
x_train,x_test,y_train,y_test=train_test_split(indep,dep,test_size=1/3,random_state=0)
sc=StandardScaler()
x_train_sc=sc.fit_transform(x_train)
x_test_sc=sc.transform(x_test)

In [4]:
def forward_selection(x_train_sc,x_test_sc,y_train,y_test,n):
    log=LogisticRegression(max_iter=1000,solver='liblinear')
    dt=DecisionTreeClassifier()
    models=[log,dt]
    best_train_x=[]
    best_test_x=[]
    best_features=[]
    for model in models:
        selector=SFS(model,k_features=n,forward=False,scoring='accuracy')
        selector.fit(x_train_sc,y_train)
        mask=selector.k_feature_names_
        feature_indices=[int(i) for i in mask]
        feature_names=dataset.columns[feature_indices]
        best_features.append(feature_names)
        x_train_fs=selector.transform(x_train_sc)
        best_train_x.append(x_train_fs)
        x_test_fs=selector.transform(x_test_sc)
        best_test_x.append(x_test_fs)
    return best_train_x,best_test_x,best_features,y_train,y_test
    

In [5]:
def accuracy(model,x_test_fs,y_test):
    y_pred=model.predict(x_test_fs)
    accu=accuracy_score(y_pred,y_test)
    return accu

In [6]:
def log(x_train_fs,y_train,x_test_fs,y_test):
    model=LogisticRegression(max_iter=1000,solver='liblinear')
    model.fit(x_train_fs,y_train)
    accuracy_log=accuracy(model,x_test_fs,y_test)
    return accuracy_log

def dt(x_train_fs,y_train,x_test_fs,y_test):
    model=DecisionTreeClassifier()
    model.fit(x_train_fs,y_train)
    accuracy_dt=accuracy(model,x_test_fs,y_test)
    return accuracy_dt

def svc(x_train_fs,y_train,x_test_fs,y_test):
    model=SVC(kernel='linear')
    model.fit(x_train_fs,y_train)
    accuracy_svc=accuracy(model,x_test_fs,y_test)
    return accuracy_svc

In [7]:
def combo(x_train_sc,x_test_sc,y_train,y_test,n):
    best_train_x,best_test_x,best_features,y_train,y_test=forward_selection(x_train_sc,x_test_sc,y_train,y_test,n)
    log_acc=[]
    dt_acc=[]
    svc_acc=[]
    #all_acc=[log_acc,dt_acc,svc_acc]
    for x_train_fs,x_test_fs in zip(best_train_x,best_test_x):
        log_mod=log(x_train_fs,y_train,x_test_fs,y_test)
        log_acc.append(log_mod)
        dt_mod=dt(x_train_fs,y_train,x_test_fs,y_test)
        dt_acc.append(dt_mod)
        svc_mod=svc(x_train_fs,y_train,x_test_fs,y_test)
        svc_acc.append(svc_mod)
    result=[log_acc,dt_acc,svc_acc]
    import pandas as pd
    score_table=pd.DataFrame(result,columns=['Logistic Regression','Decision Tree'],index=['Logistic Regression','Decision Tree','SVM'])
    features=pd.DataFrame(best_features,index=['Logistic Regression','Decision Tree'])
    return score_table,features
    #print("\nscore_table:\n",features)
    

In [8]:
combo(x_train_sc,x_test_sc,y_train,y_test,5)

(                     Logistic Regression  Decision Tree
 Logistic Regression             0.924812       0.969925
 Decision Tree                   0.954887       0.932331
 SVM                             0.939850       0.954887,
                       0     1     2           3       4
 Logistic Regression  al  hrmo  sg_c  rbc_normal  dm_yes
 Decision Tree        sc   pcv    wc        sg_c    sg_d)